In [ ]:
import mcstasscript as ms
import load_quiz
import instrument_3

quiz = load_quiz.exercise_2()

# McStasScript exercise 2 mystery instrument 3
In this sessions exercise there are three quiz notebooks, each about a mystery instrument object. The purpose is to teach how McStasScript can be used to investigate and understand an instrument in a different way than reading the code. Reading the source code will give you all the information, but when instruments gets larger it will take a significant amount of time. Try to answer all the questions without looking at the instrument code directly, though it is available in this folder. 

In [ ]:
instr = instrument_3.make_instrument() # Obtain mystery instrument object

### Question 1
What kind of instrument is described by the given instrument object?

- A: Laue Camera
- B: Triple Axis Spectrometer
- C: Diffractometer on continuous source
- D: Small Angle Neutron Scattering instrument
- E: Time of Flight Diffractometer
- F: Time of Flight Spectrometer

In [ ]:
# Use McStasScript help features to investigate the instrument
instr.show_diagram()
instr.show_parameters()

In [ ]:
quiz.instrument_3_question_1("D")

### Question 2
What sample component is used?

- A: Powder1
- B: PowderN
- C: SANS_spheres2
- D: Phonon_simple
- E: Single_crystal
- F: Incoherent
- G: Tunneling_sample

In [ ]:
# Use McStasScript features to investigate the instrument
instr.get_component("sample_conventional")

In [ ]:
quiz.instrument_3_question_2("C")

### Question 3
Does this instrument use any interesting McStas features?

- A: GROUP
- B: EXTEND
- C: WHEN
- D: JUMP
- E: SPLIT
- F: target_index

In [ ]:
# Use McStasScript features to investigate the instrument
instr.show_diagram()

In [ ]:
quiz.instrument_3_question_3(["B", "C", "E"])

### Question 4
This instrument does not have a guide, add a curved guide between the source and the collimator section with 10 segments of Guide_gravity components with m=2 and length 1.5 m. Each should be rotated 0.3 deg around the y axis relative to the former element and have a 1 cm gap to avoid overlap. Set the size of the guide to 3x3 cm^2 and start it 2 m from the source.

In [ ]:
# Since we add a guide, it is necessary to update the focus parameters of the source
source = instr.get_component("source")
source.set_parameters(focus_xw=0.03, focus_yh=0.03, dist=2)

# Consider the source and the dist the first element in order to write a simple loop
last_element = source
last_length = source.dist

for index in range(10):
    name = "guide_" + str(index) # Create a unique name for each guide elemen
    guide = instr.add_component(name, "Guide_gravity", before="coll1") # Place each element before coll1
    guide.set_parameters(l=1.5, m=2, w1=source.focus_xw, h1=source.focus_yh) # Set reasonable parameters
    guide.set_AT(last_length, RELATIVE=last_element) # Set position 
    guide.set_ROTATED([0, 0.3, 0], RELATIVE=last_element) # Set rotation
    
    last_element = guide # save the last guide element for the next loop iteration
    last_length = guide.l + 0.01 # save the length of this element for the next loop iteration
    
# Update the position of the collimation pinholes
coll1 = instr.get_component("coll1")
coll1.set_AT(last_length + 3, RELATIVE=last_element)

coll2 = instr.get_component("coll2")
coll2.set_AT(3, RELATIVE=coll1)

In [ ]:
quiz.instrument_3_question_4(instr)

### Question 5
The curved guide will introduce a wavelength cut off as neutrons with the lowest wavelengths can't be transported. Use the diagnostics features to find the wavelength where the transmission is half of the maximum.

In [ ]:
# Lets see how the curved guide cuts off the lower wavelengths
instr.set_parameters(wavelength=6, d_wavelength=5.9)
instr.show_diagram(variable="l")

In [ ]:
# To get an accurate answer we use the diagnostics tool
diag = ms.Diagnostics(instr)
diag.settings(suppress_output=True, ncount=5E6)

diag.add_point(before="guide_0")
diag.add_point(before="coll1")
diag.add_point(before="coll2")
diag.run()

In [ ]:
diag.clear_views()
# See wavelength and add lines for full and half intensity 
diag.add_view("l", axis1_values=[3.7, 4.1], axis2_values=[380, 190])
diag.add_view("l", "dx")
diag.plot()

In [ ]:
quiz.instrument_3_question_5(4.0)

# Run the instrument
Try to run the instrument with the widget interface.

In [ ]:
import mcstasscript.jb_interface as ms_widget
%matplotlib widget

In [ ]:
ms_widget.show(instr)

## Using the MCPL bridge tool
Next we try to use the MCPL bridge tool to run part of the simulation and dump the beam to a MCPL file.

### Quetion 6
Set the *run_to* point of the instrument to "sample_position" and provide the instrument object as the answer.

In [ ]:
instr.run_to("sample_position")
instr.show_run_subset()
instr.show_diagram()

In [ ]:
quiz.instrument_3_question_6(instr)

### Question 7
Set the number of rays to 2E7 and run the instrument using the backengine method. This will generate a beamdump which will be tracked by the persistent database linked to the instrument name.

In [ ]:
instr.settings(suppress_output=True, ncount=1E7)
instr.backengine()
instr.show_dumps() # Show an overview over the created beam dumps

In [ ]:
quiz.instrument_3_question_7(instr)

### Question 8
Set the instrument to run from the sample_position to the end of the instrument and give the instrument object as the answer. This will use the latest beamdump at the sample_position as a starting point for the simulation.

In [ ]:
# First reset the run_to to the end of the instrument
instr.run_to(None)
# Then set run_from to sample_position to load the available beamdump
instr.run_from("sample_position")

# Show the status of the instrument
instr.show_run_subset()
instr.show_diagram()

In [ ]:
quiz.instrument_3_question_8(instr)

## Run the instrument from the beam dump
Use the widget interface to run the instrument, notice that parameters that affect anything before the sample no longer has any effect, but the sample parameters still affect the outcome. The ncount can not be changed through the interface, as it uses all rays stored in the file. The simulation is also much faster as the guide and collimation is skipped.

In [ ]:
ms_widget.show(instr)